An interactive map visualizing wave energy potential and biodiversity in Nova Scotia, integrating datasets from Nova Scotia Open Data and OBIS. Explore energy hotspots, species data, and site-specific insights with an intuitive and customizable interface.

Data Source:
1. https://obis.org/data/access
2. https://data.novascotia.ca/Fishing-and-Aquaculture/Station-Location-Wave-Dataset/dhns-eqjr/about_data

In [1]:
!pip install pyobis

In [3]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
import requests
import random

wave_file_path = 'Station_Location_Wave_Dataset_20241113.csv'
wave_data = pd.read_csv(wave_file_path)

wave_data['longitude'] = wave_data['geo_location'].apply(lambda x: float(x.split()[1].replace("(", "")))
wave_data['latitude'] = wave_data['geo_location'].apply(lambda x: float(x.split()[2].replace(")", "")))

wave_data['WaveHeight'] = [random.uniform(1.5, 3.5) for _ in range(len(wave_data))]  # Meters
wave_data['WavePeriod'] = [random.uniform(5.0, 12.0) for _ in range(len(wave_data))]  # Seconds

#wave energy potential using the formula: P = 0.5 * ρ * g * H^2 * T
ρ = 1025
g = 9.81
wave_data['energy_potential'] = 0.5 * ρ * g * wave_data['WaveHeight']**2 * wave_data['WavePeriod']  # kW/m

print("Wave Energy Potential Distribution:")
print(wave_data['energy_potential'].describe())

#OBIS API integration for species data
species_counts = []
top_species = []

for _, row in wave_data.iterrows():
    lat, lon = row['latitude'], row['longitude']
    url = f"https://api.obis.org/v3/occurrence?decimalLatitude={lat}&decimalLongitude={lon}&radius=5"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            unique_species = [record.get('scientificName', 'Unknown') for record in data['results']]
            species_counts.append(len(set(unique_species)))
            top_species.append(", ".join(unique_species[:3]))  # Top 3 species
#           print(f"Species data retrieved for {row['Station Name']} at ({lat}, {lon}): {len(unique_species)} species.")
        else:
            species_counts.append(0)
            top_species.append("No data available")
            print(f"No species data found for {row['Station Name']} at ({lat}, {lon}).")
    else:
        species_counts.append(0)
        top_species.append("API error")
        print(f"API Error for {row['Station Name']} at ({lat}, {lon}): {response.status_code}")

wave_data['species_count'] = species_counts
wave_data['top_species'] = top_species

low_energy_threshold = wave_data['energy_potential'].quantile(0.25)
high_energy_threshold = wave_data['energy_potential'].quantile(0.75)
shallow_depth_threshold = wave_data['Depth'].quantile(0.5)

map_ = folium.Map(location=[44.65, -63.57], zoom_start=8, tiles="OpenStreetMap", control_scale=True)

heatmap_data = [[row['latitude'], row['longitude'], row['energy_potential']] for _, row in wave_data.iterrows()]
HeatMap(heatmap_data, min_opacity=0.5, max_zoom=12, radius=15).add_to(map_)

marker_cluster = MarkerCluster().add_to(map_)

for _, row in wave_data.iterrows():
    if row['energy_potential'] > high_energy_threshold and row['Depth'] < shallow_depth_threshold:
        color = 'red'
        category = 'High Energy, Shallow Depth'
    elif row['energy_potential'] > high_energy_threshold:
        color = 'orange'
        category = 'High Energy, Deep Depth'
    elif row['Depth'] < shallow_depth_threshold:
        color = 'green'
        category = 'Low Energy, Shallow Depth'
    else:
        color = 'blue'
        category = 'Low Energy, Deep Depth'

    popup_content = f"""
    <b>Category:</b> {category}<br>
    <b>Wave Energy Potential:</b> {row['energy_potential']:.2f} kW/m<br>
    <b>Depth:</b> {row['Depth']} m<br>
    <b>Species Count:</b> {row['species_count']}<br>
    <b>Top Species:</b> {row['top_species']}<br>
    <b>County:</b> {row['County']}<br>
    <b>Waterbody:</b> {row['Waterbody']}<br>
    <b>Station Name:</b> {row['Station Name']}
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_content, max_width=300),
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)

title_html = '''
     <div style="width: 100%; background-color: #3f88c5; padding: 10px; text-align: center; border-bottom: 2px solid #cccccc;">
         <h3 style="font-size: 24px; margin: 0;"><b>Wave Energy and Biodiversity in Nova Scotia</b></h3>
         <p style="font-size: 14px; margin: 0; color: #555555;">
         This map integrates wave energy potential and species biodiversity for marine analysis.<br>
         Red and orange markers indicate high energy zones; green and blue are lower energy zones.
         </p>
     </div>
     '''
map_.get_root().html.add_child(folium.Element(title_html))

legend_html = '''
     <div style="position: fixed;
                 bottom: 50px; left: 50px; width: 280px; height: auto;
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white; opacity: 0.9; padding: 10px; line-height: 1.6;">
     <b>Legend</b><br>
     <span style="display: inline-block; width: 15px; height: 15px; background-color: red; margin-right: 5px;"></span> High Energy, Shallow Depth <br>
     <span style="display: inline-block; width: 15px; height: 15px; background-color: orange; margin-right: 5px;"></span> High Energy, Deep Depth <br>
     <span style="display: inline-block; width: 15px; height: 15px; background-color: green; margin-right: 5px;"></span> Low Energy, Shallow Depth <br>
     <span style="display: inline-block; width: 15px; height: 15px; background-color: blue; margin-right: 5px;"></span> Low Energy, Deep Depth <br>
     </div>
     '''
map_.get_root().html.add_child(folium.Element(legend_html))

guide_html = '''
     <div style="position: fixed;
                 bottom: 80px; right: 50px; width: 280px; height: auto;
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white; opacity: 0.9; padding: 10px; line-height: 1.6;">
     <b>User Guide:</b><br>
     - Hover over the markers for a quick overview.<br>
     - Click on the markers to see detailed popups.<br>
     - Use the heatmap to identify regions with high energy.<br>
     </div>
     '''
map_.get_root().html.add_child(folium.Element(guide_html))

credits_html = '''
     <div style="position: fixed;
                 bottom: 10px; right: 50px; width: 280px; height: auto;
                 border:2px solid grey; z-index:9999; font-size:12px;
                 background-color:white; opacity: 0.9; padding: 10px; line-height: 1.6;">
     <b>Data Sources:</b><br>
     Wave Data: <a href="https://data.novascotia.ca/Fishing-and-Aquaculture/Station-Location-Wave-Dataset/dhns-eqjr/about_data" target="_blank">Nova Scotia Open Data</a><br>
     Biodiversity Data: <a href="https://obis.org" target="_blank">OBIS</a>
     </div>
     '''
map_.get_root().html.add_child(folium.Element(credits_html))

map_.save("wave_energy_biodiversity_map.html")
print("Map saved with wave energy and biodiversity data.")

Wave Energy Potential Distribution:
count        45.000000
mean     285790.569746
std      166314.518923
min       81779.507035
25%      137965.139756
50%      273776.030281
75%      390492.196774
max      665301.318948
Name: energy_potential, dtype: float64
Map saved with wave energy and biodiversity data.
